In [41]:
import pyblp
import numpy as np
import pandas as pd
import statsmodels.api as sm

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

# Load the data 
sales_data = pd.read_csv("OTC_Sales.csv", header=1)
print(sales_data.head())
demo_data = pd.read_csv("OTC_Demographics.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'OTC_Sales.csv'

In [ ]:
#Identify the columns that contain the product sales for each product 
product_cols = sales_data.columns[3:14]

#QUESTION 1
#Now, well start the calculations for the Sum Stats Table. 
# --------------------------------------------------------------#
#Market Shares 
#Note- for this table I want the market shares to be meaningful in terms of one another (headache medicine). 
# Convert all product columns to numeric only 
sales_data[product_cols] = sales_data[product_cols].apply(pd.to_numeric, errors='coerce')

# Total sales
total_sales_all_products = sales_data[product_cols].sum().sum()
market_share = {}

# Calculate market share for each product
for product in product_cols:
    total_sales_product = sales_data[product].sum()  # Sum of sales for the product
    market_share[product] = total_sales_product / total_sales_all_products  # Calculate market share

for i, product in enumerate(product_cols, 1):
    print(f"Market Share of Product_{i}: {market_share[product]:.2%}")
# --------------------------------------------------------------#


Market Share of Product_1: 14.49%
Market Share of Product_2: 17.97%
Market Share of Product_3: 11.67%
Market Share of Product_4: 11.88%
Market Share of Product_5: 7.81%
Market Share of Product_6: 3.59%
Market Share of Product_7: 4.06%
Market Share of Product_8: 3.48%
Market Share of Product_9: 8.00%
Market Share of Product_10: 9.37%
Market Share of Product_11: 7.69%


In [18]:
#Unit Price (price per 1 bottle of a product)
price_cols = sales_data.columns[15:26]
unit_price = {}
sales_data[price_cols] = sales_data[price_cols].apply(pd.to_numeric, errors='coerce')

unit_price = sales_data[price_cols].mean()
for i, product in enumerate(price_cols, 1):
    print(f"Average Unit Price of Product_{i}: {unit_price[product]:.2f}")




Average Unit Price of Product_1: 4.95
Average Unit Price of Product_2: 7.03
Average Unit Price of Product_3: 2.97
Average Unit Price of Product_4: 5.15
Average Unit Price of Product_5: 8.16
Average Unit Price of Product_6: 2.67
Average Unit Price of Product_7: 3.62
Average Unit Price of Product_8: 3.97
Average Unit Price of Product_9: 1.94
Average Unit Price of Product_10: 4.43
Average Unit Price of Product_11: 2.19


In [19]:
#price per 100 tablets 
price_per_100 = {}

#load in the number of tablets per product
tablets_per_bottle = {
    1: 25, 2: 50, 3: 100,  # Tylenol
    4: 25, 5: 50, 6: 100,  # Advil
    7: 25, 8: 50, 9: 100,  # Bayer
    10: 50, 11: 100        # Store Brand
}

for i in range(1, 12):  # Products 1 to 11
    price_column = price_cols[i-1]  # Use the existing list of price column names
    price_per_100[i] = (unit_price[price_column] / tablets_per_bottle[i]) * 100

for product, price in price_per_100.items():
    print(f"Price per 100 tablets of Product_{product}: {price:.2f}")


Price per 100 tablets of Product_1: 19.79
Price per 100 tablets of Product_2: 14.06
Price per 100 tablets of Product_3: 2.97
Price per 100 tablets of Product_4: 20.58
Price per 100 tablets of Product_5: 16.32
Price per 100 tablets of Product_6: 2.67
Price per 100 tablets of Product_7: 14.48
Price per 100 tablets of Product_8: 7.94
Price per 100 tablets of Product_9: 1.94
Price per 100 tablets of Product_10: 8.85
Price per 100 tablets of Product_11: 2.19


In [20]:
#unit wholesale price (this will be the avg cost of each product)
unit_price_wholesale = {}
cost_cols = sales_data.columns[26:37]
sales_data[cost_cols] = sales_data[cost_cols].apply(pd.to_numeric, errors='coerce')

unit_price_wholesale = sales_data[cost_cols].mean()

for i, product in enumerate(cost_cols, 1):
    print(f"Average Unit Wholesale Price of Product_{i}: {unit_price_wholesale[product]:.2f}")

Average Unit Wholesale Price of Product_1: 3.68
Average Unit Wholesale Price of Product_2: 5.77
Average Unit Wholesale Price of Product_3: 2.03
Average Unit Wholesale Price of Product_4: 3.63
Average Unit Wholesale Price of Product_5: 6.10
Average Unit Wholesale Price of Product_6: 1.85
Average Unit Wholesale Price of Product_7: 2.44
Average Unit Wholesale Price of Product_8: 3.71
Average Unit Wholesale Price of Product_9: 0.91
Average Unit Wholesale Price of Product_10: 1.91
Average Unit Wholesale Price of Product_11: 0.03


In [ ]:
# QUESTION 2
# part a
#To calculate market share Sij, im going to calculate the total market size. Sij will be the number who buy product i in market j /market size
sales_data["total_market"] = sales_data["count"]
for product in product_cols:
    sales_data[f"share_{product}"] = sales_data[product] / sales_data["total_market"]

# Calculate outside option share, S0j (1 - sum of all product shares)
sales_data["outside_share"] = 1 - sales_data[[f"share_{p}" for p in product_cols]].sum(axis=1)

# Log transformed shares(log(s_ij) - log(s_0j))
for product in product_cols:
    sales_data[f"logit_share_{product}"] = np.log(sales_data[f"share_{product}"]) - np.log(sales_data["outside_share"])
